# Introduction to yabadaba: Interact with databases

Once you have defined Record classes for your data, you can use yabadaba's Database objects to interact with databases to query and alter the contained records.

In [1]:
# Standard Python libraries
import datetime

# Import the main yabadaba package
import yabadaba

# Import the yabadaba_demo package that defines the Records to use
import yabadaba_demo

# Show yabadaba version
print('yabadaba version =', yabadaba.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

yabadaba version = 0.3.0
Notebook executed on 2024-12-18


## 1. Check the modular components

Once you have imported yabadaba (and any record-defining packages) you can check the status of the various modular components by calling yabadaba.check_modules() or the check_styles() method of the various module managers.

In [2]:
yabadaba.check_modules()

Database styles that passed import:
- local: <class 'yabadaba.database.LocalDatabase.LocalDatabase'>
- mongo: <class 'yabadaba.database.MongoDatabase.MongoDatabase'>
- cdcs: <class 'yabadaba.database.CDCSDatabase.CDCSDatabase'>
Database styles that failed import:


Record styles that passed import:
- FAQ: <class 'yabadaba_demo.record.FAQ.FAQ'>
- album: <class 'yabadaba_demo.record.album.Album.Album'>
Record styles that failed import:
- bad_record: <class 'ModuleNotFoundError'>: No module named 'package_that_does_not_exist'


Query styles that passed import:
- bool_match: <class 'yabadaba.query.BoolMatchQuery.BoolMatchQuery'>
- str_contains: <class 'yabadaba.query.StrContainsQuery.StrContainsQuery'>
- str_match: <class 'yabadaba.query.StrMatchQuery.StrMatchQuery'>
- list_contains: <class 'yabadaba.query.ListContainsQuery.ListContainsQuery'>
- int_match: <class 'yabadaba.query.IntMatchQuery.IntMatchQuery'>
- float_match: <class 'yabadaba.query.FloatMatchQuery.FloatMatchQuery'>
- date_mat

We can see that currently there are three Database styles defined by yabadaba.

In [3]:
yabadaba.databasemanager.check_styles()

Database styles that passed import:
- local: <class 'yabadaba.database.LocalDatabase.LocalDatabase'>
- mongo: <class 'yabadaba.database.MongoDatabase.MongoDatabase'>
- cdcs: <class 'yabadaba.database.CDCSDatabase.CDCSDatabase'>
Database styles that failed import:



Three Record styles were also found that were defined in yabadaba_demo.  These three styles are

- The FAQ style defined in the previous Notebook.  The only difference is that the yabadaba_demo version has values specified for the xsl_filename and xsd_filename properties.
- The demo style that is used in subsequent Notebooks to demonstrate how to define Records for more complex schemas.
- The bad_record style that fails import because it tries to import a non-existent package.

In [4]:
yabadaba.recordmanager.check_styles()

Record styles that passed import:
- FAQ: <class 'yabadaba_demo.record.FAQ.FAQ'>
- album: <class 'yabadaba_demo.record.album.Album.Album'>
Record styles that failed import:
- bad_record: <class 'ModuleNotFoundError'>: No module named 'package_that_does_not_exist'



## 2. Interacting with databases

Everything is now in place to interact with databases.  The Database classes are managed using the databasemanager in a fashion similar to how the recordmanager handles Records.  The main difference being that some Database styles are already defined meaning that you (hopefully) won't need to define your own Database classes.  

__NOTE__: For this demo we will be interacting with the "local" style database of JSON files contained within the "yabadaba_demo_database" folder located in the same directory as this Notebook.  It contains the three FAQ records that were defined at the beginning of the previous Notebook. 

### 2.1. Initializing databases

Database objects are initialized by providing all of the necessary access parameters for the specific database.  As these vary between the different styles of databases, the allowed parameters also vary.  

#### 2.1.1. Local database 

A local style Database exists as a local directory containing individual JSON and XML files, as well as csv cache files containing the compiled metadata associated with the hosted records.  The cache files greatly increase the speed of queries as it allows for parsing the records without reading and loading all of the individual files.  A local-style database is initialized with the following parameters

- __host__ (*str*) The host name (local directory path) for the database.
- __format__ (*str, optional*) The format that the model records are saved as.  Can be either JSON or XML.  Default value is JSON.
- __indent__ (*int or None, optional*) The indentation used when saving the records.  If None (default) then the saved records are compact.  Otherwise, the lines in the file will be indented by multiples of this value based on the model's element recursion.  Compact records are smaller, while indented records are easier to read.

In [5]:
database = yabadaba.load_database(style='local', host='yabadaba_demo_database')
print(database)

database style local at /home/lmh1/Python-packages/yabadaba/doc/yabadaba_demo_database


#### 2.1.2. Mongo database 

A mongo style Database interacts with a MongoDB instance. The initialization parameters relate to the corresponding parameters of the pymongo.Mongoclient class as it is utilized by the Database class.

- __host__ (*str*) The mongo host to connect to.  Default value is 'localhost'.
- __port__ (*int*) Then port to use in connecting to the mongo host.  Default value is 27017.
- __database__ (*str*) The name of the database in the mongo host to interact with. Default value is 'iprPy'
- __\*\*kwargs__ (*dict, optional*) Any extra keyword arguments needed to initialize a pymongo.MongoClient object.

#### 2.1.3. CDCS database

A cdcs Database style interacts with a CDCS (Configurable Data Curation System) instance.  CDCS databases provide REST APIs meaning that the initialization parameters correspond to web requests permissions.

- __host__ (*str*) The host name (url) for the database.
- __username__ (*str or tuple of two str*) The username to use for accessing the database.  Alternatively, a tuple of (username, password).
- __password__ (*str, optional*) The password associated with username to use for accessing the database. This can either be the password as a str, or a str path to a file containing only the password. If not given, a prompt will ask for the password.
- __cert__ (*str, optional*) if String, path to ssl client cert file (.pem). If Tuple, ('cert', 'key') pair.
- __certification__ (*str, optional*) Alias for cert. Retained for compatibility.
- __verify__ (*bool or str, optional*) Either a boolean, in which case it controls whether we verify the server's TLS certificate, or a string, in which case it must be a path to a CA bundle to use. Defaults to True.
- __cdcsversion__ (*str, optional*) For CDCS versions 2.X.X, this allows for specifying the full CDCS version to ensure the class methods perform the correct REST calls.  This can be specified as "#.#.#", or if None is given will default to "2.15.0".  For CDCS versions 3.X.X, this is ignored as version info is obtained directly from the database.

### 2.2. Querying records from a database

There are a few different methods for querying data in a database.  These methods are for the most part universal across the different database styles although individual database styles can contain extra parameters specific to their infrastructure.

- __get_records()__ returns a numpy.NDArray containing Record objects that match the query. Adding the parameter return_df=True will also return the pandas.DataFrame that get_records_df() returns.
- __get_record()__ returns a single Record object if exactly one matching record is found.  Otherwise it will issue an error.
- __get_records_df()__ returns a pandas.DataFrame of Record metadata for all records that match the query.
- __count_records()__ returns an int count of the number of records that match the query.  This is usually much faster if you only want to know how many matching records there are.

Parameters for the query methods are comparable
        
- __style__ (*str, optional*) The record style to search. If not given, a prompt will ask for it.
- __\*\*kwargs__ (*any, optional*) Any extra options specific to the database style or metadata search parameters specific to the record style.

In [6]:
# Count how many FAQ records are in the database
database.count_records(style='FAQ')

3

In [7]:
# Get all records of FAQ as FAQ objects
database.get_records(style='FAQ')

array([<yabadaba_demo.record.FAQ.FAQ object at 0x7f9319315ed0>,
      dtype=object)

In [8]:
# Get pandas.DataFrame of all FAQ record simple metadata values 
database.get_records_df(style='FAQ')

,name,question,answer
0,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
1,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
2,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


In [9]:
# Get the single FAQ record that has 'bear' in its question
record = database.get_record(style='FAQ', question='bear')
print(record.model.json(indent=4))

{
    "faq": {
        "question": "Fuzzywuzzy was a bear. Fuzzywuzzy had no hair. So Fuzzywuzzy wasn't fuzzy, was he?",
        "answer": "Nope."
    }
}


For the most part, the list of queryable terms corresponds to the values that were defined for the Record class.  You can view a short description of the allowed query parameters by looking at the querydoc parameter of one record of the given style.

In [10]:
yabadaba.querydoc('FAQ', render=True)

# FAQ Query Parameters

- __question__ (*str or list, optional*): Return only the records where question contains the given values
- __answer__ (*str or list, optional*): Return only the records where answer contains the given values


Additionally, there is a convenience method retrieve_record() which gets a matching record from the database and saves it to a local file all at once.  This can be used to easily fetch and save a record so that external software can interact with it.
    
- __style__ (*str, optional*) The record style to search. If not given, a prompt will ask for it.
- __dest__ (*path, optional*) The parent directory where the record will be saved to.  If not given, will use the current working directory.
- __format__ (*str, optional*) The file format to save the record in: 'json' or 'xml'.  Default is 'json'.
- __indent__ (*int, optional*) The number of space indentation spacings to use in the saved record for the different tiered levels.  Default is 4.  Giving None will create a compact record.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __\*\*kwargs__ (*any, optional*) Any extra options specific to the database style or metadata search parameters specific to the record style.

### 2.3. Adding/updating records

Records can be added/updated to a database using the add_record() and update_record() Database methods.  Many of the database operations in yabadaba work on the principle that all records of a given style should have unique names.  The two methods adhere to that according to the following operations
- __add_record()__ will add a new record to the database as long as no similarly named record exists.  If a record with the same style and name is already in the database, then the method will throw an error.
- __update_record()__ will update the contents of an existing record in the database.  If no record in the database has the matching name and style, then the method will throw an error.

A combined add/update can easily be done with code that does try: add, except: update.

Both methods take similar parameters

- __record__ (*Record, optional*) The new record to add to the database.  If not given, then name, style and content are required.
- __style__ (*str, optional*) The record style for the new record.  Required if record is not given.
- __name__ (*str, optional*) The name to assign to the new record.  Required if record is not given.
- __model__ (*str or DataModelDict, optional*) The model contents of the new record.  Required if record is not given.
- __build__ (*bool, optional*) If True, then the uploaded content will be (re)built based on the record's attributes.  If False (default), then record's existing content will be loaded if it exists, or built if it doesn't exist.
- __verbose__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

In [15]:
# Create two new FAQ records like was done in the previous Notebook
init_faq = yabadaba.load_record(style='FAQ', name='init', question='Can I assign values during init?', answer='Yes, you can!')

build_faq = yabadaba.load_record(style='FAQ')
build_faq.name = 'build'
build_faq.question = 'Is it easy to build record contents by assigning to object attributes?'
build_faq.answer = 'It seems that way to me.'

In [16]:
# Add the new records to the database.  Note that we use build=True as we did not call build_model beforehand!!
database.add_record(init_faq, build=True, verbose=True)
database.add_record(build_faq, build=True, verbose=True)

FAQ record named init added to /home/lmh1/Python-packages/yabadaba/doc/yabadaba_demo_database
FAQ record named build added to /home/lmh1/Python-packages/yabadaba/doc/yabadaba_demo_database


In [17]:
database.get_records_df('FAQ')

,name,question,answer
0,build,Is it easy to build record contents by assigni...,It seems that way to me.
1,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
2,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
3,init,Can I assign values during init?,"Yes, you can!"
4,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


If we now modify a record, we can update the database's copy with update_record().

__NOTE__: For local-style databases, if you change record contents you will need to include the refresh_cache=True option on the next get query.  This is because the cache is a csv file that by default only adds or deletes lines when the associated record files are added or deleted. Using refresh_cache=True will rebuild the csv file from the contents of the contained files.

In [19]:
build_faq.answer = 'Yes, and you can modify them later.'
build_faq.build_model()

database.update_record(build_faq)
database.get_records_df('FAQ', refresh_cache=True)

,name,question,answer
0,build,Is it easy to build record contents by assigni...,"Yes, and you can modify them later."
1,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
2,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
3,init,Can I assign values during init?,"Yes, you can!"
4,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


### 2.4. Deleting records

The __delete_record()__ method will delete a single record.  The method's parameters make it possible to specify which record to delete.

- __record__ (*Record, optional*) The record to delete from the database.  If not given, name and/or style are needed to uniquely define the record to delete.
- __style__ (*str, optional*) The style of the record to delete.
- __name__ (*str, optional*) The name of the record to delete.

In [20]:
database.delete_record(style='FAQ', name='init')
database.delete_record(style='FAQ', name='build')
database.get_records_df('FAQ')

,name,question,answer
0,faq,What does a FAQ Record represent?,A frequently asked question and the correspond...
1,fuzzy,Fuzzywuzzy was a bear. Fuzzywuzzy had no hair....,Nope.
2,woodchuck,How much wood would a woodchuck chuck if a woo...,A woodchuck would chuck as much wood as a wood...


## 3. Interacting with non-record files

The current yabadaba code contains features for interacting with non-record files contained within the same database according to the following scheme:

- Each record entry in the database has a unique style + name combination.
- Each record can have a .tar.gz assigned to it that collects all associated data files.  These tar.gz files are stored in the associated databases as blobs.
- The tar data can then be uniquely accessed using the record's style and name.

This is the design choice of the original code from which yabadaba grew out of. Additional methods are planned to be added at a later time to streamline the access of any generic blob from the database or from an external location.   

### 3.1. Database operations

The tar files are managed in a similar way as the records with get_tar(), add_tar(), update_tar(), and delete_tar() methods. All four methods allow for unique identification of the associated record by either passing in the record object or by providing the style and name separately. 

- get_tar() retrieves the tar file associated with a record from the database and returns it as a tarfile object (default) or as raw bytes data (if raw=True setting is used).
- add_tar() adds a new tar file to the database for the associated record.  The contents to archive can either be specified as raw bytes data with the "tar" parameter or by specifying a "root_dir" as the basis for creating a new tar.gz file.
- update_tar() updates the tar file associated with a record.  The contents to archive can be specified in the same way as add_tar().
- delete_tar() deletes the tar file associated with a record from the database. 

### 3.2. Record operations

Each record object has some additional attributes and methods that supports easily accessing the data stored in the record's associated tar file.

- Record.database is an attribute that can be set to point to a yabadaba Database object.  When a record is retrieved using one of the get methods above, this value is automatically set for each record to the database that was queried.  Having each record aware of its hosting database makes it possible to design dynamic methods for the record classes that rely on other database contents.
- Record.tar is the tarfile representation of the record's associated tar file.  This is dynamically built when Record.tar is first accessed by calling get_tar() using the Record's database value.
- Record.get_file() will retrieve and extract a single file from the record's tar according to a given file name. Calling this will build Record.tar if necessary.
- Record.display_image() extends get_file() to automatically display an image loaded from the tar file if working in an IPython environment like a Jupyter Notebook.
- Record.clear_tar() will close the tarfile object and reset the Record.tar file. Using this can possibly help with memory issues if interacting with lots of record tars depending on how useful Python's garbage collectors feel like being.

## 4. Other utility methods

The Database class also defines additional methods that perform more complex convenience operations.

### 4.1. copy_records()

Copies records from the current database to another database.

- __dest__ (*Database*) The destination database to copy records to.
- __record_style__ (*str, optional*) The record style to copy.  If record_style and records not given, then the available record styles will be listed and the user prompted to pick one.  Cannot be given with records.
- __records__ (*list, optional*) A list of Record objects from the current database to copy to dest.  Allows the user full control on which records to copy/update.  Cannot be given with record_style.
- __includetar__ (*bool, optional*) If True, the tar archives will be copied along with the records. If False, only the records will be copied. (Default is True).
- __overwrite__ (*bool, optional*) If False (default) only new records and tars will be copied. If True, all existing content will be updated.

### 4.2. destroy_records()

Permanently deletes multiple records and their associated tars all at once.
        
- __record_style__ (*str, optional*) The record style to delete.  If given, all records of that style will be deleted. If neither record_style nor records given, then the available record styles will be listed and the user prompted to pick one.
- __records__ (*list, optional*) A list of pre-selected records to delete. 